In [507]:
import pandas as pd
import numpy as np 
import os
import tabula
import requests
import PyPDF2
import re

In [508]:
from tabula import read_pdf

In [509]:
root_directory = 'C:\\Users\Katri\OneDrive\Desktop\College_Board_SAT_Data\data'


In [510]:
df = pd.DataFrame(columns=['Year','State',"Total_number_of_HS_students","Participation_rate",'Demo_Group', 'Num_Test_Takers','Ave_Score', 'ERW_Ave_Score', 'Math_Ave_Score',
       'Percent_met_both_benchmarks', 'Percent_met_no_benchmark',
       'Percent_Pass_ERW', 'Percent_Pass_Math'])

In [511]:
for subdir, dirs, files in os.walk(root_directory):
    # Loop through the files in each directory
    for file in files:
        year = extract_year(str(file))
        #Figure out what state the data is for 
        state= extract_state_name(str(file))
        #Figure out on what pages the relevant data is stored
        filename = os.path.join(subdir, file)
        page_1, page_2 = find_pages_with_phrases(filename)
        #Convert to table 
        table = tabula.read_pdf(filename,pages=page_2, multiple_tables=True,silent=True, area=[99.0, 9.0, 730, 589.0])
        if len(table)>0:
            #Figure out how many test takes there are 
            test_takers_total = table[0].loc[table[0]["Total"] == "Total", "Test Takers"].values[0]
            n = total_number_of_hs_students(filename,page_1,test_takers_total)
            #Change the entries with no_response
            no_response(table[0])
            #convert to a pandas data frame 
            df = df.append(page_five(table[0],state,year,n))


In [512]:
df.head()

,Year,State,Total_number_of_HS_students,Participation_rate,Demo_Group,Num_Test_Takers,Ave_Score,ERW_Ave_Score,Math_Ave_Score,Percent_met_both_benchmarks,Percent_met_no_benchmark,Percent_Pass_ERW,Percent_Pass_Math
0,2017,AL,49102,4.873529,Total,2393,1165.0,593.0,572.0,62.0,12.0,86.0,64.0
1,2017,AL,49102,4.873529,Took Essay1,887,1226.0,621.0,605.0,72.0,7.0,91.0,74.0
2,2017,AL,49102,4.873529,American Indian/Alaska Native,15,1049.0,538.0,511.0,47.0,27.0,73.0,47.0
3,2017,AL,49102,4.873529,Asian,199,1272.0,608.0,664.0,78.0,6.0,84.0,87.0
4,2017,AL,49102,4.873529,Black/African American,501,992.0,513.0,479.0,26.0,35.0,64.0,28.0


In [518]:
output_dir = './Output_data'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Export the dataframe to a CSV file in the output directory
output_file = os.path.join(output_dir, 'output_file.csv')
df.to_csv(output_file, index=False)

In [520]:

output_file = os.path.join(output_dir, 'output_file.xlsx')
df.to_excel(output_file, index=False)

In [416]:
def total_number_of_hs_students(file,page,test_takers_total):
    def delete_string_from_list(lst, string_to_delete):
        return [s for s in lst if s != string_to_delete]
    def find_next_element(lst):
        # Find the index of the element containing "SAT", "Participation", and "Rate"
        index = -1
        for i, s in enumerate(lst):
            if "SAT" in s and "Participation" in s and "Rate" in s:
                index = i
                break
        if index == -1:
            return None  # The target string is not found

        # Find the first element AFTER the target string that does NOT contain a %
        for i in range(index+1, len(lst)):
            s = lst[i]
            if "%" not in s:
                return int(s.replace(",", ""))
        return None  # No matching element is found
    
    with open(file, 'rb') as pdf_file:
                # Create a PyPDF2 PdfFileReader object
                pdf_reader = PyPDF2.PdfFileReader(pdf_file)
                page = pdf_reader.getPage(page-1)
                text = page.extractText()
                text = text.replace('\t', ' ') 
                text= text.split("\n")
    return find_next_element(delete_string_from_list(text,test_takers_total))

In [506]:
#This function takes the table from page 5 of the report, cleans the data and turns it into a dataframe 
def page_five(df,state,year,n):
    df.dropna(subset=[list(df.columns)[0]],inplace=True)
    df.dropna(axis=1,how='all',inplace=True)
    df.reset_index(drop = True)
    df.columns = ["Demo_Group","Num_Test_Takers","Ave_Score","ERW_Ave_Score","Math_Ave_Score","Percent_met_both_benchmarks",
              "Percent","Percent_met_no_benchmark"]
    
    df[['Percent_Pass_ERW', 'Percent_Pass_Math']] = df['Percent'].str.split(' ', expand=True)
    df.drop(columns=['Percent'], inplace=True)
    df = df[~df['Num_Test_Takers'].str.contains('Test Takers')].copy()
    df["Num_Test_Takers"] = df["Num_Test_Takers"].str.split().str[0]
    
    cols_to_convert = ["Percent_met_both_benchmarks", "Percent_met_no_benchmark", "Percent_Pass_ERW", 
                       "Percent_Pass_Math","Ave_Score","ERW_Ave_Score","Math_Ave_Score"]
    for col in cols_to_convert:
        # Replace the '%' character with an empty string
        df[col] = df[col].str.replace('%', '')
        # Convert the values to integers
        df[col] = pd.to_numeric(df[col], errors='coerce')
        
    df["Num_Test_Takers"] = df["Num_Test_Takers"].str.replace(",","")
    df["Num_Test_Takers"] = pd.to_numeric(df["Num_Test_Takers"], downcast='integer')
    
    df["State"] = state
    df["Year"] = year
    df["Total_number_of_HS_students"] = n 
    df = df.reset_index(drop=True)
    total_hs_students = df.at[0, 'Total_number_of_HS_students']
    num_test_takers = df.at[0, 'Num_Test_Takers']
    df['Participation_rate'] = (num_test_takers / total_hs_students) * 100
    df['Demo_Group'] = df['Demo_Group'].replace('Native Hawaian/Other Pacific Islander', 'Native Hawaiian/Other Pacific Islander')
    df = df[~df['Demo_Group'].isin(['Used at any point', 'Did not use'])]
    df = df[['Year','State',"Total_number_of_HS_students","Participation_rate",'Demo_Group', 'Num_Test_Takers','Ave_Score', 'ERW_Ave_Score', 'Math_Ave_Score',
       'Percent_met_both_benchmarks', 'Percent_met_no_benchmark',
       'Percent_Pass_ERW', 'Percent_Pass_Math']]
    num_columns = ['Total_number_of_HS_students', 'Participation_rate', 'Num_Test_Takers', 'Ave_Score', 'ERW_Ave_Score',
       'Math_Ave_Score', 'Percent_met_both_benchmarks',
       'Percent_met_no_benchmark', 'Percent_Pass_ERW', 'Percent_Pass_Math']
    for col in num_columns:
        # Convert the values to integers
        df[col] = pd.to_numeric(df[col], errors='coerce')

    return df

In [431]:
def no_response(df):
    mapping = {
    'Two or More Races': 'Race: No Response',
    'Another Language': 'Language: No Response',
    'Graduate Degree': 'Parental Education: No Response',
    'Male': 'Gender: No Response'}
    # Iterate over the Demo_Group column and update the "No Response" entries
    prev_entry = None
    for i, entry in df[df.columns[0]].iteritems():
        if entry == 'No Response' or entry == "Another/ No Response":
            if prev_entry in mapping:
                df.at[i, df.columns[0]] = mapping[prev_entry]
        else:
            prev_entry = entry

In [419]:
#This function takes the pdf file and extracts from it the number 
#of high school students in the state that year. 
def helper_function(filename,page_1):
    def find_next_element(lst):
        # Find the index of the element containing "SAT", "Participation", and "Rate"
        index = -1
        for i, s in enumerate(lst):
            if "SAT" in s and "Participation" in s and "Rate" in s:
                index = i
                break
        if index == -1:
            return None  # The target string is not found

        # Find the first element AFTER the target string that does NOT contain a %
        for i in range(index+1, len(lst)):
            s = lst[i]
            if "%" not in s:
                return int(s.replace(",", ""))
        return None  # No matching element is found
    if type(page_1)==int:
        with open(filename, 'rb') as pdf_file:
                # Create a PyPDF2 PdfFileReader object
                pdf_reader = PyPDF2.PdfFileReader(pdf_file)
                page = pdf_reader.getPage(page_1-1)
                text = page.extractText()
                text = text.split("\n")
                num_students = find_next_element(text)
        return num_students
    else: return None 

In [420]:
#Write a helper function that helps you find on what pages the correct tables are. 
def find_pages_with_phrases(pdf_file_path):
    pdf_file = open(pdf_file_path, 'rb')
    pdf_reader = PyPDF2.PdfFileReader(pdf_file)

    page_1_keywords = ['Suite', 'Participation', 'Summary', 'NMSQT']
    page_2_keywords = ['Participation', 'Performance', 'Hawaiian', 'Benchmark', 'Female']
    page_1 = None
    page_2 = None

    for page in range(pdf_reader.numPages):
        text = pdf_reader.getPage(page).extractText()

        # Check if all page 1 keywords are in the text
        if all(keyword in text for keyword in page_1_keywords):
            page_1 = page + 1

        # Check if all page 2 keywords are in the text
        if all(keyword in text for keyword in page_2_keywords):
            page_2 = page + 1

        # If we've found both pages, we can break out of the loop early
        if page_1 is not None and page_2 is not None:
            break

    pdf_file.close()
    
    return page_1, page_2

In [421]:
#This function takes in the file name in a string format and extracts the year. 
def extract_year(input_string):
    # Define the regular expression pattern to match four-digit years
    pattern = r"\d{4}"

    # Search for the pattern in the input string
    match = re.search(pattern, input_string)

    # Check if a match was found
    if match:
        # Get the matched string (the four-digit year)
        year = match.group()

        # Return the year
        return int(year)
    else:
        # Return None if no four-digit year was found in the input string
        return None

In [488]:
#This function takes in the file name in a string format and extracts the year.

def extract_state_name(input_string):
    # Define a list of USA state names
    states = ['alabama','alaska','arizona','arkansas','california', 'colorado', 'connecticut', 'delaware', 'florida', 'georgia',
 'hawaii', 'idaho', 'illinois','indiana','iowa','kansas','kentucky','louisiana','maine',
 'maryland', 'massachusetts','michigan','minnesota','mississippi','missouri','montana','nebraska',
 'nevada','new-hampshire','new-jersey','new-mexico','new-york','north-carolina','north-dakota','ohio','oklahoma',
 'oregon', 'pennsylvania', 'rhode-island', 'south-carolina', 'south-dakota', 'tennessee', 'texas', 'utah',
 'vermont','virginia', 'washington', 'west-virginia', 'wisconsin', 'wyoming', 'puerto-rico', 'virgin-islands', 'district-columbia']
    def state_name_to_code(state_name):
        state_codes = {
        'alabama': 'AL', 'alaska': 'AK', 'arizona': 'AZ', 'arkansas': 'AR', 'california': 'CA',
        'colorado': 'CO', 'connecticut': 'CT', 'delaware': 'DE', 'florida': 'FL', 'georgia': 'GA',
        'hawaii': 'HI', 'idaho': 'ID', 'illinois': 'IL', 'indiana': 'IN', 'iowa': 'IA', 'kansas': 'KS',
        'kentucky': 'KY', 'louisiana': 'LA', 'maine': 'ME', 'maryland': 'MD', 'massachusetts': 'MA',
        'michigan': 'MI', 'minnesota': 'MN', 'mississippi': 'MS', 'missouri': 'MO', 'montana': 'MT',
        'nebraska': 'NE', 'nevada': 'NV', 'new-hampshire': 'NH', 'new-jersey': 'NJ', 'new-mexico': 'NM',
        'new-york': 'NY', 'north-carolina': 'NC', 'north-dakota': 'ND', 'ohio': 'OH', 'oklahoma': 'OK',
        'oregon': 'OR', 'pennsylvania': 'PA', 'rhode-island': 'RI', 'south-carolina': 'SC', 'south-dakota': 'SD',
        'tennessee': 'TN', 'texas': 'TX', 'utah': 'UT', 'vermont': 'VT', 'virginia': 'VA', 'washington': 'WA',
        'west-virginia': 'WV', 'wisconsin': 'WI', 'wyoming': 'WY', 'puerto-rico': 'PR', 'virgin-islands': 'VI',
        'district-columbia': 'DC', "district-of-columbia":"DC"}
        return state_codes.get(state_name, None)
    # Define a regular expression pattern to match USA state names
    
    pattern = r'\b(' + '|'.join(states) + r')\b'

    # Search for the pattern in the input string
    match = re.search(pattern, input_string, re.IGNORECASE)

    # Check if a match was found
 
    if match:
        # Get the matched string (the state name)
        state_name = match.group()
        state = state_name_to_code(state_name )
        # Return the state name
        return state
    else:
        # Return None if no state name was found in the input string
        return None